In [96]:
import json
import pandas as pd
from os.path import join
from scipy.io import savemat
import numpy as np
import itertools
from surgeon_recording.reader import Reader
from scipy.io import savemat
from sklearn.preprocessing import MinMaxScaler

In [2]:
reader = Reader()

In [3]:
datafolder = join('..', 'data', 'cuts')

In [80]:
exp_folders = [join(datafolder, 'cut' + str(i)) for i in (np.arange(5) + 2)]

In [81]:
exp_folders

['../data/cuts/cut2',
 '../data/cuts/cut3',
 '../data/cuts/cut4',
 '../data/cuts/cut5',
 '../data/cuts/cut6']

In [82]:
def insert_row(data, row):
    return data.append(pd.Series(row), ignore_index=True)

In [83]:
def downsample(data, time_vector):
    current_time_index = 0
    downsampled_data = pd.DataFrame(columns=data.columns)
    for i in range(len(time_vector)):
        t = time_vector.iloc[i]
        start_time = current_time_index
        while data['relative_time'].iloc[current_time_index] < t:
            current_time_index = current_time_index + 1
        stop_time = current_time_index
        average_data = data.iloc[start_time:stop_time, :].mean()
        downsampled_data = insert_row(downsampled_data, average_data)
    return downsampled_data

In [108]:
def preprocess_motion(data, frames_list, scaler):
    column_list = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z') for f in frames_list))
    data[column_list] = 2 * scaler.fit_transform(data[column_list]) - 1

In [109]:
def preprocess_emg(data, emg_list, scaler):
    # first take the absolute value of the data
    data[emg_list] = data[emg_list].abs()
    # then apply a minmax scaler
    data[emg_list] = 2 * scaler.fit_transform(data[emg_list]) - 1

In [110]:
frames_list = ['blade', 'wrist']
optitrack_list = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z',
                                                     f + '_qx', f + '_qy', f + '_qz', f + '_qw')
                                                    for f in frames_list))
emg_list = ['emg' + str(i) for i in range(8)]

In [111]:
timeseries = []

scalers = {}
scalers['optitrack'] = MinMaxScaler()
scalers['emg'] = MinMaxScaler()

for folder in exp_folders:
    reader.play(folder)
    # extract optitrack data and preprocess them
    opt_data = reader.data['optitrack'].reset_index(drop=True)
    preprocess_motion(opt_data, frames_list, scalers['optitrack'])
    # downsample emg and preprocess it
    emg_data = downsample(reader.data['emg'], opt_data['relative_time'])
    preprocess_emg(emg_data, emg_list, scalers['emg'])
    # merge the data
    merge_data = pd.concat([opt_data, emg_data.iloc[:, 3:]], axis=1)
    # store in the list
    timeseries.append(merge_data)

In [113]:
merged_column_list = optitrack_list

In [114]:
# drop the data in matlab
matdata = {}
matdata['timeseries'] = []
for t in timeseries:
    # drop the index and times columns and convert to numpy array
    matdata['timeseries'].append(t[merged_column_list].to_numpy())

In [115]:
savemat("segmentation_data.mat", matdata)